In [12]:
#importing required modules
from __future__ import print_function, division
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import time
import os
import copy

import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision import datasets, transforms as T
from torchvision import models
from efficientnet_pytorch import EfficientNet
import os
import torch.optim as optim
from PIL import ImageFile
from sklearn.metrics import accuracy_score
import sklearn.metrics
from sklearn.metrics import confusion_matrix, classification_report

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
#declaring batch size
batch_size = 32

#applying required transformations on the dataset
img_transforms = {
    'train':
    T.Compose([
        T.Resize(size=(224,224)), 
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
        ]),

    'valid':
    T.Compose([
        T.Resize(size=(224,224)),
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),

    'test':
    T.Compose([
        T.Resize(size=(224,224)),
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
     }

#creating Location of data: train, validation, test
data='./data/'

train_path=os.path.join(data,'train')
valid_path=os.path.join(data,'test')
test_path=os.path.join(data,'valid')


#creating Datasets to each of  folder created in prev
train_file=datasets.ImageFolder(train_path,transform=img_transforms['train'])
valid_file=datasets.ImageFolder(valid_path,transform=img_transforms['valid'])
test_file=datasets.ImageFolder(test_path,transform=img_transforms['test'])


#creating loaders for the dataset
loaders_transfer={
    'train':torch.utils.data.DataLoader(train_file,batch_size,shuffle=True),
    'val':torch.utils.data.DataLoader(valid_file,batch_size,shuffle=True),
    'test': torch.utils.data.DataLoader(test_file,batch_size,shuffle=True)
}

dataset_sizes = {
    'train': len(train_file),
    'val': len(valid_file),
    'test': len(test_file)
}

class_names = train_file.classes

In [3]:
use_cuda = torch.cuda.is_available()

In [4]:
if use_cuda:  
    dev = "cuda" 
else:  
    dev = "cpu"  
device = torch.device(dev) 
print(device)

cuda


In [5]:
#importing the pretrained resnet18 model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in loaders_transfer[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2939 Acc: 0.8761
val Loss: 0.1467 Acc: 0.9452

Epoch 1/24
----------
train Loss: 0.0746 Acc: 0.9775
val Loss: 0.0350 Acc: 0.9956

Epoch 2/24
----------
train Loss: 0.0373 Acc: 0.9902
val Loss: 0.0279 Acc: 0.9956

Epoch 3/24
----------
train Loss: 0.0276 Acc: 0.9935
val Loss: 0.0116 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.0153 Acc: 0.9981
val Loss: 0.0247 Acc: 0.9934

Epoch 5/24
----------
train Loss: 0.0126 Acc: 0.9977
val Loss: 0.0184 Acc: 0.9934

Epoch 6/24
----------
train Loss: 0.0082 Acc: 0.9995
val Loss: 0.0168 Acc: 0.9956

Epoch 7/24
----------
train Loss: 0.0060 Acc: 1.0000
val Loss: 0.0173 Acc: 0.9934

Epoch 8/24
----------
train Loss: 0.0065 Acc: 0.9991
val Loss: 0.0143 Acc: 0.9956

Epoch 9/24
----------
train Loss: 0.0058 Acc: 0.9995
val Loss: 0.0148 Acc: 0.9956

Epoch 10/24
----------
train Loss: 0.0073 Acc: 0.9991
val Loss: 0.0139 Acc: 0.9956

Epoch 11/24
----------
train Loss: 0.0057 Acc: 0.9991
val Loss: 0.0132 Acc: 0.9956

Ep

In [8]:
# Defining the test function

def test(loaders, model, criterion, use_cuda):

    # monitoring test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    preds = []
    targets = []

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # moving to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # updating average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # converting the output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        preds.append(pred)
        targets.append(target)
        # compare predictions
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
    
    return preds, targets

# calling test function
preds, targets = test(loaders_transfer, model_ft, criterion, use_cuda)

In [9]:
#converting the tensor object to a list for metric functions

preds2, targets2 = [],[]

for i in preds:
      for j in range(len(i)):
        preds2.append(i.cpu().numpy()[j])
for i in targets:
      for j in range(len(i)):
        targets2.append(i.cpu().numpy()[j])

In [10]:
#Computing the accuracy
acc = accuracy_score(targets2, preds2)
print("Accuracy: ", acc)

Accuracy:  0.9912280701754386


In [13]:
perf_report = classification_report(preds2, targets2)
print(perf_report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       228
           1       0.99      0.99      0.99       228

    accuracy                           0.99       456
   macro avg       0.99      0.99      0.99       456
weighted avg       0.99      0.99      0.99       456

